In [20]:
import camelot
data = camelot.read_pdf('timetable.pdf', pages='6-25')

In [21]:
import pandas
datadf = data[0].df
for i in range(1,20):
    datadf = datadf.append(data[i].df)
datadf

,0,1,2,3,4,5,6,7,8,9
0,COM \nCODE,CLASS \nNUMBER \nAS IN ERP,COURSENO,COURSETITLE,L P U,CREDIT \nSEC,STAT,INSTRUCTOR IN CHARGE/Instructor,DAYS/ H,COMPRE DATE
1,021112,,1011 BIO F211,BIOLOGICAL CHEMISTRY,3 0 3,L,1,RAVIPRASAD ADURI/ Meenal Kowshik/ Rajesh Mehrotra,T TH S 3,09/12/20 (FN)
2,021112,,1012 BIO F211,BIOLOGICAL CHEMISTRY,3 0 3,T,1,Raviprasad Aduri/ Meenal Kowshik/ Rajesh Mehrotra,T 10,
3,021114,,1241 BIO F212,MICROBIOLOGY,3 1 4,L,1,JUDITH MARIA BRAGANCA/ Utpal Roy/ Vijayashree ...,T TH S 4,03/12/20 (FN)
4,021114,,1242 BIO F212,MICROBIOLOGY,3 1 4,P,1,"Judith Maria Braganca, Utpal Roy, Vijayashree ...",T TH 9,
5,021113,,1013 BIO F213,CELL BIOLOGY,3 0 3,L,1,ANGSHUMAN SARKAR/ Indrani Talukdar,M W F 3,05/12/20 (AN)
6,021113,,1014 BIO F213,CELL BIOLOGY,3 0 3,T,1,Angshuman Sarkar/ Indrani Talukdar,T 8,
7,021115,,1015 BIO F214,INTEGRATED BIOLOGY,3 0 3,L,1,SUMIT BISWAS/ Arnab Banerjee/ Rajesh Mehrotra,M W F 2,07/12/20 (FN)
8,021115,,1016 BIO F214,INTEGRATED BIOLOGY,3 0 3,T,1,Sumit Biswas/ Arnab Banerjee/ Rajesh Mehrotra,F 10,
9,021651,,1154 BIO F266,STUDY PROJECT,3,R,1,SRIKANTH MUTNURI,,


In [22]:
df = datadf.copy(deep=True)
df = df.drop(index=[0]).drop(columns=[0,1]).reset_index(drop=True)
df = df.rename(columns={2:'course_code', 3:'course_name', 4:'units', 5:'ltp', 6:'sec', 7:'instructor', 8:'time', 9:'compre'})
df['course_code'] = df['course_code'].apply(lambda x : x.split()[1]+'_'+x.split()[2])

In [23]:
df = df.dropna(subset=['time'])
df.iloc[pandas.Index(df['time']).get_loc('T TH 5-6:30 PM')]['time'] = 'T TH 10 11'
df.iloc[pandas.Index(df['time']).get_loc('TH S 12-1:30 PM')]['time'] = 'TH S 5 6'

In [24]:
days = {'M':-1, 'T':11, 'W':23, 'TH':35, 'F':47, 'S':59}

def reduce(code):
    codes = [code]
    for i in range(len(code)-1):
        if code[i] not in days and code[i+1] in days:
            codes = [(code[0:i+1])] + reduce(code[i+1:])
    return codes

def decode(codes):
    time = [0] * 72
    for code in codes:
        hr = []
        dy = []
        for i in code:
            if i in days:
                dy.append(i)
            else:
                hr.append(i)             
        for d in dy:
            for h in hr:
                time[days[d] + int(h)] = 1
    return time

In [25]:
df['time'] = df['time'].apply(lambda code : decode(reduce(code.split())))

In [26]:
df['instructor'] = df['instructor'].apply(lambda name : name.replace('/',',').replace('\n','').upper())

In [27]:
def merge_sec(sec):
    ans = sec[['course_name','units','ltp','instructor','time','compre']].iloc[0].copy(deep=True)
    for s in range(1,len(sec)):
        ans['ltp'] += sec.iloc[s]['ltp']
        ans['instructor'] += ', ' + sec.iloc[s]['instructor']
        ans['time'] = [ans['time'][i] or sec.iloc[s]['time'][i] for i in range(72)]
    return ans

In [28]:
df = df.groupby(['course_code','sec']).apply(merge_sec)

In [29]:
df1 = df.copy(deep=True)

In [30]:
df1 = df1.reset_index(['course_code','sec'])

In [31]:
datadict = dict()
def make_dict(crs):
    
    secs = dict()
    for s in crs.index :
        sec = {
            'instructor' : crs['instructor'][s],
            'time' : crs['time'][s],
            'compre' : crs.iloc[0]['compre']
        }
        secs[crs['sec'][s]] = sec
        
    details = {
        'name' : crs.iloc[0]['course_name'],
        'units' : crs.iloc[0]['units'],
        'ltp' : crs.iloc[0]['ltp'],
        'secnos' : len(secs),
        'sec' : secs
    }
    
    datadict[crs.iloc[0]['course_code']] = details

In [32]:
df1.groupby('course_code').apply(make_dict)

""


In [33]:
datadict

{'BIOT_F346': {'name': 'GENOMICS',
  'units': '3 0 3',
  'ltp': 'L',
  'secnos': 1,
  'sec': {'1': {'instructor': 'TUSAR TIRTHA SAHA, SANDHYA MEHROTRA',
    'time': [0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0],
    'compre': '12/12/20 (AN)'}}},
 'BIO_F211': {'name': 'BIOLOGICAL CHEMISTRY',
  'units': '3 0 3',
  'ltp': 'LT',
  'secnos': 1,
  'sec': {'1': {'instructor': 'RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA, RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA'

In [34]:
import json
with open("datadict.json", 'w') as f :
    json.dump(datadict, f)